### Реквизиты автора: Рябов Михаил Сергеевич

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import pickle
import mlflow
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from mlflow.models import infer_signature
from sklearn.model_selection import GridSearchCV
from multiprocessing import Process

In [46]:
# Настройка MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Student_models")

<Experiment: artifact_location='mlflow-artifacts:/712008823741102884', creation_time=1743593768251, experiment_id='712008823741102884', last_update_time=1743593768251, lifecycle_stage='active', name='Student_models', tags={}>

In [62]:
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    
    df['Extracurricular Activities'] = df['Extracurricular Activities'].map({'Yes': 1, 'No': 0})
    
    df = df[(df['Hours Studied'] <= 24) & 
            (df['Sleep Hours'] >= 4) & 
            (df['Previous Scores'] >= 40)]
    
    return df

def prepare_data(df):
    X = df.drop(columns=['Performance Index'])
    y = df['Performance Index']
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    return X_train, X_val, y_train, y_val

In [60]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [52]:
def create_preprocessor():
    numeric_features = ['Hours Studied', 'Previous Scores', 
                      'Sleep Hours', 'Sample Question Papers Practiced',
                      'Extracurricular Activities']
    
    preprocessor = Pipeline(
        steps=[('scaler', StandardScaler())]
    )
    return preprocessor

In [54]:
def train_and_log_model(model, params, model_name, X_train, y_train, X_val, y_val):
    with mlflow.start_run(run_name=model_name):
        preprocessor = create_preprocessor()
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        if params:
            mlflow.log_params({
                "grid_search_params": str(params)
            })
            
            clf = GridSearchCV(pipeline, params, cv=5, scoring='r2')
            clf.fit(X_train, y_train)
            best_model = clf.best_estimator_
            
            best_params = {"best_" + k: v for k, v in clf.best_params_.items()}
            mlflow.log_params(best_params)
        else:
            pipeline.fit(X_train, y_train)
            best_model = pipeline
        
        y_pred = best_model.predict(X_val)
        rmse, mae, r2 = eval_metrics(y_val, y_pred)
        
        mlflow.log_metrics({
            "rmse": rmse,
            "mae": mae,
            "r2": r2
        })
        
        signature = infer_signature(X_train, best_model.predict(X_train))
        mlflow.sklearn.log_model(best_model, model_name, signature=signature)
        
        return rmse, mae, r2

In [56]:
# Датасеет: https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression
# Загрузка датасета и начало работы
df = load_and_preprocess_data("Student_Performance.csv")
X_train, X_val, y_train, y_val = prepare_data(df)

# SGDRegressor
sgd_params = {
    'model__alpha': [0.0001, 0.001, 0.01],
    'model__l1_ratio': [0.15, 0.2, 0.25],
    'model__max_iter': [1000, 2000]
}
sgd_metrics = train_and_log_model(
    SGDRegressor(random_state=42),
    sgd_params,
    "SGD_Regressor",
    X_train, y_train, X_val, y_val
)

# RandomForest
rf_params = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [5, 10, None],
    'model__min_samples_split': [2, 5]
}
rf_metrics = train_and_log_model(
    RandomForestRegressor(random_state=42),
    rf_params,
    "Random_Forest",
    X_train, y_train, X_val, y_val
)

# GradientBoosting
gb_params = {
    'model__n_estimators': [50, 100],
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3, 5]
}
gb_metrics = train_and_log_model(
    GradientBoostingRegressor(random_state=42),
    gb_params,
    "Gradient_Boosting",
    X_train, y_train, X_val, y_val
)

# Сравнение результатов
results = {
    "SGD_Regressor": dict(zip(["rmse", "mae", "r2"], sgd_metrics)),
    "Random_Forest": dict(zip(["rmse", "mae", "r2"], rf_metrics)),
    "Gradient_Boosting": dict(zip(["rmse", "mae", "r2"], gb_metrics))
}

print("\nСравнение моделей:")
for model, metrics in results.items():
    print(f"\n{model}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

best_model = max(results.items(), key=lambda x: x[1]["r2"])
print(f"\nЛучшая модель: {best_model[0]} по R2 - {best_model[1]['r2']:.4f}")

C:\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run SGD_Regressor at: http://127.0.0.1:5000/#/experiments/712008823741102884/runs/a14e393c5c974e9fac91e292b4660a53
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/712008823741102884


C:\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Random_Forest at: http://127.0.0.1:5000/#/experiments/712008823741102884/runs/82663077f3524079af4a73ecf08c3969
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/712008823741102884


C:\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Gradient_Boosting at: http://127.0.0.1:5000/#/experiments/712008823741102884/runs/fdb99fa7cd8d44d5a13d9c194dbfeb3e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/712008823741102884

Сравнение моделей:

SGD_Regressor:
rmse: 2.0213
mae: 1.6115
r2: 0.9890

Random_Forest:
rmse: 2.1692
mae: 1.7276
r2: 0.9873

Gradient_Boosting:
rmse: 2.0892
mae: 1.6578
r2: 0.9882

Лучшая модель: SGD_Regressor по R2 - 0.9890
